In [1]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import getpass
password = getpass.getpass()

In [2]:
connection_string = 'mysql+pymysql://root:' + password +'@localhost/bank'
engine = create_engine(connection_string)

In [3]:
import mysql.connector

In [4]:
cnx = mysql.connector.connect(user = 'root',password = password, host = '127.0.0.1')
cnx.is_connected()

True

In [5]:
cursor = cnx.cursor()
cursor.execute("select * from sakila.rentals_may")
result = pd.DataFrame(cursor.fetchall())
result.columns = [head[0] for head in cursor.description]
result.head()

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-16 00:30:53
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,2006-02-16 00:30:53
2,3,2005-05-24 23:03:39,1711,408,2005-06-01 22:12:39,1,2006-02-16 00:30:53
3,4,2005-05-24 23:04:41,2452,333,2005-06-03 01:43:41,2,2006-02-16 00:30:53
4,5,2005-05-24 23:05:21,2079,222,2005-06-02 04:33:21,1,2006-02-16 00:30:53


In [15]:
cursor.execute("SELECT customer_id, COUNT(rental_id) as num_rentals FROM sakila.rentals_may GROUP BY customer_id")
result_may = pd.DataFrame(cursor.fetchall())
result_may.columns = [head[0] for head in cursor.description]
result_may.head()

,customer_id,num_rentals
0,1,2
1,2,1
2,3,2
3,5,3
4,6,3


In [11]:
cursor2 = cnx.cursor()
cursor.execute("select * from sakila.rentals_june")
result = pd.DataFrame(cursor.fetchall())
result.columns = [head[0] for head in cursor.description]
result.head()

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1158,2005-06-14 22:53:33,1632,416,2005-06-18 21:37:33,2,2006-02-16 00:30:53
1,1159,2005-06-14 22:55:13,4395,516,2005-06-17 02:11:13,1,2006-02-16 00:30:53
2,1160,2005-06-14 23:00:34,2795,239,2005-06-18 01:58:34,2,2006-02-16 00:30:53
3,1161,2005-06-14 23:07:08,1690,285,2005-06-21 17:12:08,1,2006-02-16 00:30:53
4,1162,2005-06-14 23:09:38,987,310,2005-06-23 22:00:38,1,2006-02-16 00:30:53


In [27]:
cursor2.execute("SELECT customer_id, COUNT(rental_id) as num_rentals FROM sakila.rentals_june GROUP BY customer_id")
result_june = pd.DataFrame(cursor2.fetchall())
result_june.columns = [head[0] for head in cursor2.description]
result_june.head()

,customer_id,num_rentals
0,1,7
1,2,1
2,3,4
3,4,6
4,5,5


In [48]:
def compare_rentals(result_may, result_june):
    merged_df = pd.merge(result_may.rename(columns={'num_rentals': 'num_rentals_may'}),
                         result_june.rename(columns={'num_rentals': 'num_rentals_june'}),
                         on='customer_id', how='outer')
    merged_df['diference'] = merged_df['num_rentals_june'] - merged_df['num_rentals_may']
    return merged_df


In [49]:
result_df = compare_rentals(result_may, result_june)
print(result_df)


     customer_id  num_rentals_may  num_rentals_june  diference
0              1              2.0                 7        5.0
1              2              3.0                 1       -2.0
2              3              1.0                 4        3.0
3              4              NaN                 6        NaN
4              5              NaN                 5        NaN
..           ...              ...               ...        ...
585          595              NaN                 2        NaN
586          596              NaN                 2        NaN
587          597              NaN                 3        NaN
588          598              NaN                 1        NaN
589          599              NaN                 4        NaN

[590 rows x 4 columns]
